In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
from os.path import join as opj
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pylab
import os
from matplotlib import pyplot
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten, Activation
from keras.layers import GlobalMaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import Concatenate
from keras.models import Model
from keras import initializers
from keras.optimizers import Adam
from keras.optimizers import rmsprop
from keras.layers.advanced_activations import LeakyReLU, PReLU
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping,ReduceLROnPlateau

from keras.datasets import cifar10
from keras.applications.inception_v3 import InceptionV3
from keras.applications.vgg16 import VGG16
from keras.layers import Concatenate, Dense, LSTM, Input, concatenate
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss
import collections

Using TensorFlow backend.


In [2]:
train = pd.read_json("/".join(os.getcwd().split("/")[:-1])+"/Data/train.json")
test = pd.read_json("/".join(os.getcwd().split("/")[:-1])+"/Data/test.json")

In [3]:
test['inc_angle']=pd.to_numeric(test['inc_angle'], errors='coerce')
train = train[train['inc_angle']!='na']
train['inc_angle']=pd.to_numeric(train['inc_angle'], errors='coerce')#We have only 133 NAs.
train.index = [i for i in range(len(train.index))]

In [44]:
X_band_test_1=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_1"]])
X_band_test_2=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_2"]])
X_band_test_3=(X_band_test_1+X_band_test_2)/2
test1 = np.concatenate([X_band_test_1[:, :, :, np.newaxis], X_band_test_2[:, :, :, np.newaxis], X_band_test_3[:, :, :, np.newaxis]], axis=-1)

In [6]:
bar1 = 39
angle_1 = train[train['inc_angle'] <= bar1]
angle_2 = train[train['inc_angle'] > bar1]
train_angle_split = [angle_1,angle_2]

In [45]:
index1 = test[test['inc_angle']<=bar1].index
index2 = test[test['inc_angle']>bar1].index
tt = [(test1[index1],index1),(test1[index2],index2)]

In [8]:
def data_engineer(train):
    
    y_train=train['is_iceberg']
    
    #Generate the training data
    X_band_1=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_1"]])
    X_band_2=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_2"]])
    X_band_3=(X_band_1+X_band_2)/2
    X_train = np.concatenate([X_band_1[:, :, :, np.newaxis], X_band_2[:, :, :, np.newaxis], X_band_3[:, :, :, np.newaxis]], axis=-1)
    return X_train,np.array(y_train),y_train.index


    

In [13]:
#get early_stopping and best_weight_save
def get_callbacks(filepath):
    es = EarlyStopping('val_loss', patience=13, mode="min")
    msave = ModelCheckpoint(filepath,monitor='val_loss', save_best_only=True)
    reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, verbose=0, epsilon=1e-4, mode='min')
    return [es, msave,reduce_lr_loss]


In [10]:
datagen = ImageDataGenerator(horizontal_flip = True,vertical_flip = True)
def gen_flow(X1, y,batch_size=64):
    genX1 = datagen.flow(X1,y,  batch_size=batch_size,seed=55)
    while True:
            X1i = genX1.next()
            yield X1i

In [39]:
#get vgg model
def VGG():
    base_model = VGG16(include_top=False, input_shape=X_train.shape[1:])
    x = base_model.get_layer('block5_pool').output
    
    x = GlobalMaxPooling2D(name = "glob_max_pool")(x)
    merge_one = Dense(128, activation='relu', name='fc2')(x)
    merge_one = Dropout(0.3,name = "drop1")(merge_one)
    merge_one = Dense(32, activation='relu', name='fc3')(merge_one)
    merge_one = Dropout(0.3, name = "drop2")(merge_one)
    
    predictions = Dense(1, activation='sigmoid')(merge_one)
    
    model = Model(input=base_model.input, output=predictions)   
    sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='binary_crossentropy',
                  optimizer=sgd)
    return model


In [54]:
def k_fold(X_train,y_train,X_test,epochs = 100, k=3):
    folds = list(StratifiedKFold(n_splits=k, shuffle=True, random_state=16).split(X_train, y_train))
    y_valid_pred = np.array([0.0]*len(X_train))
    y_test_pred = 0
    for j, (train_idx, valid_idx) in enumerate(folds):
        X_train_cv = X_train[train_idx]
        y_train_cv = y_train[train_idx]
        X_valid_cv = X_train[valid_idx]
        y_valid_cv= y_train[valid_idx]
        

        #define file path and get callbacks
        file_path = "%s_aug_model_weights.hdf5"%j
        callbacks = get_callbacks(filepath=file_path)
        gen_fl = gen_flow(X_train_cv,y_train_cv,batch_size=64)
        vgg16 = VGG()

        vgg16.fit_generator(
                gen_fl,
                steps_per_epoch=12,
                epochs=epochs,
                shuffle=True,
                verbose=0,
                validation_data=(X_valid_cv, y_valid_cv),
                callbacks=callbacks)

        #Getting the Best Model
        vgg16.load_weights(filepath=file_path)

        #train predictions
        #y_train_pred = vgg16.predict(X_train_cv)[:,0]
        #print('Train loss: ' +str(log_loss(y_train_cv,y_train_pred)))

        #validation predictions
        y_valid_pred[valid_idx]=vgg16.predict(X_valid_cv)[:,0]
        #print('Validation loss: ' +str(log_loss(y_valid_cv,y_valid_pred[valid_idx])))
        
        #Getting Test predictions
        y_test_pred += vgg16.predict(X_test)[:,0]

    print("Half Validation Loss: "+str(log_loss(y_train, y_valid_pred)))
    y_test_pred = y_test_pred.reshape((y_test_pred.shape[0]))
    y_test_pred/=k;
    return y_valid_pred,y_test_pred

In [55]:
n = 30
submission = pd.DataFrame()
submission['id']=test['id']

In [ ]:
loss = []
for i in range(n):
    validation = np.array([0.0]*len(train))
    pred = np.array([0.0]*len(test))
    for i,angle in enumerate(train_angle_split):
        X_train,y_train,index = data_engineer(angle)
        X_test,index_test = tt[i]
        validation[index],pred[index_test] = k_fold(X_train,y_train,X_test,epochs = 60)
    
    #loss
    temp_loss = log_loss(train['is_iceberg'].values,validation)
    print("\nTotal Validation loss:"+str(temp_loss)+"\n")    
    loss.append(temp_loss)
    
    #submission
    submission['is_iceberg']=pred.reshape((pred.shape[0]))
    submission.to_csv("/".join(os.getcwd().split("/")[:-1])+"/Angle_Result/sub"+str(i)+".csv", index=False)
    


/home/zw720/pyenv/py3.5-gpu/lib/python3.5/site-packages/ipykernel_launcher.py:14: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("de..., inputs=Tensor("in...)`
  


Half Validation Loss: 0.216024247251
Half Validation Loss: 0.307894119647

Total Validation loss:0.26473838761

Half Validation Loss: 0.170503736495
Half Validation Loss: 0.269539502092

Total Validation loss:0.223017602685

Half Validation Loss: 0.265773233406
Half Validation Loss: 0.322371652493

Total Validation loss:0.295784631699

Half Validation Loss: 0.174647203278


In [ ]:
len(loss)

In [ ]:
loss = np.array(loss)
loss.mean()